# Motif Enrichment
- The motif hits matrix save the max loglikelyhood score of a motif in that DMR
- Before run the motif enrichment, I filter the scores for each motif using cutoff defined by
    - `np.quantile(motif_score[motif_score > 0]) * mask_quantile_to99`
- This filter will remove hits with low score, which is likely to be false positive
- Its likely boosting the significance of real enrichment

In [1]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pybedtools
import scipy.stats as stats
import seaborn as sns
from scipy.sparse import csr_matrix
from statsmodels.stats.multitest import multipletests
import pathlib

In [2]:
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = 'IT-L23+ACA'

motif_hits_path = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad'
dmr_hits_path = f'/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/{group}.DMS1.bed'
output_dir = '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment'


In [3]:
# Parameters
mask_quantile_to99 = 0.9
or_cutoff = 1.5
neg_lgp_cutoff = 10
group = "IT-L23+ACA"
motif_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifScan.h5ad"
dmr_hits_path = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/HypoDMR/IT-L23+ACA.DMS1.bed"
output_dir = "/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_by_layer/IT-L23/MotifEnrichment"


In [4]:
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## DMR hits

In [5]:
dmr_bed_df = pd.read_csv(dmr_hits_path, sep='\t', header=None, index_col=3)
dmr_bed_df.head()

,0,1,2
3,,,
DGm3c_3,chr1,3057752,3057873
DGm3c_13,chr1,3117711,3117711
DGm3c_40,chr1,3273196,3273196
DGm3c_43,chr1,3297316,3297413
DGm3c_49,chr1,3322440,3322583


In [6]:
motif_hits = anndata.read_h5ad(motif_hits_path)
motif_hits = motif_hits[dmr_bed_df.index, :].copy()
motif_ids = motif_hits.var_names
motif_hits.X = motif_hits.X.todense()
motif_hits

AnnData object with n_obs × n_vars = 35039 × 719 
    obs: 'chrom', 'start', 'end'

In [7]:
# mask small motif scores
motif_cutoff = pd.Series(np.apply_along_axis(lambda i: np.quantile(i[i>0], 0.99) * mask_quantile_to99, 
                                             0, motif_hits.X),
                         index=motif_hits.var_names)
# only keep value larger than the cutoff for each motif
motif_hits.X = np.multiply(motif_hits.X, (motif_hits.X > motif_cutoff.values[None, :]))

## Background Hits

In [8]:
background_motif_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/BackgroundDMR/MotifScan.h5ad'
)

In [9]:
dmr_bed = pybedtools.BedTool().from_dataframe(dmr_bed_df)
bg_bed = pybedtools.BedTool().from_dataframe(
    background_motif_hits.obs.reset_index().iloc[:, [1, 2, 3, 0]])

In [10]:
# exclude background that overlap with DMR
bg_no_overlap = bg_bed.intersect(dmr_bed, v=True)
use_bg = bg_no_overlap.to_dataframe().iloc[:, -1].values
background_motif_hits = background_motif_hits[use_bg, :]

# make sure col in same order
background_motif_hits = background_motif_hits[:, motif_ids].copy()

In [11]:
judge_matrix = csr_matrix(background_motif_hits.X > motif_cutoff.values[None, :])
background_motif_hits.X = background_motif_hits.X.multiply(judge_matrix)

In [12]:
background_motif_hits

AnnData object with n_obs × n_vars = 340989 × 719 
    obs: 'chrom', 'start', 'end'

## Motif hits contingency table

In [13]:
# calculate motif occurence, not considering hits here
pos = (motif_hits.X > 0).sum(axis=0)
pos_total = motif_hits.shape[0]

neg = (background_motif_hits.X > 0).sum(axis=0)
neg_total = background_motif_hits.shape[0]

In [14]:
tables = {}
for motif, _pos, _neg in zip(motif_ids, pos.A1, neg.A1):
    table = [[_pos, pos_total - _pos], [_neg, neg_total - _neg]]
    tables[motif] = table

In [15]:
results = {}
with ProcessPoolExecutor(20) as executor:
    fs = {}
    for motif, t in tables.items():
        f = executor.submit(stats.fisher_exact, t, alternative='greater')
        fs[f] = motif

    for f in as_completed(fs):
        motif = fs[f]
        odds, p = f.result()
        results[motif] = {'oddsratio': odds, 'p_value': p}
motif_enrich_df = pd.DataFrame(results).T

_, p, _, _ = multipletests(motif_enrich_df['p_value'], method='fdr_bh')
motif_enrich_df['adj_p'] = p

motif_enrich_df['-lgp'] = -np.log10(motif_enrich_df['adj_p']).replace(
    -np.inf, -300)

records = {}
for motif, t in tables.items():
    tp, tn = t[0]
    fp, fn = t[1]
    tp_rate = tp / pos_total
    fp_rate = fp / neg_total
    records[motif] = dict(tp=tp,
                          tn=tn,
                          fp=fp,
                          fn=fn,
                          tp_rate=tp_rate,
                          fp_rate=fp_rate)
counts = pd.DataFrame(records).T
motif_enrich_df = pd.concat([motif_enrich_df, counts], axis=1, sort=True)

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
motif_enrich_df['Group'] = group

In [17]:
# final filter
filtered_motif_df = motif_enrich_df[(motif_enrich_df['oddsratio'] > or_cutoff)
                                    &
                                    (motif_enrich_df['-lgp'] > neg_lgp_cutoff)]
filtered_motif_df.shape[0]

28

In [18]:
motif_enrich_df.to_msgpack(output_dir / f'{group}.motif_enrichment.msg')

/home/hanliu/miniconda3/envs/clustering/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.


In [19]:
pybedtools.cleanup()

In [20]:
filtered_motif_df

,oddsratio,p_value,adj_p,-lgp,tp,tn,fp,fn,tp_rate,fp_rate,Group
MA0091.1,1.963527,1.877983e-35,4.091728e-34,33.388093,476.0,34563.0,2375.0,338614.0,0.013585,0.006965,IT-L23+ACA
MA0095.2,1.594264,6.416671e-12,9.415483e-11,10.026157,278.0,34761.0,1702.0,339287.0,0.007934,0.004991,IT-L23+ACA
MA0143.4,1.648813,2.614224e-87,1.105663e-85,84.956377,2156.0,32883.0,13041.0,327948.0,0.061531,0.038245,IT-L23+ACA
MA0461.2,3.712647,7.506707e-239,1.349331e-236,235.869882,1112.0,33927.0,2984.0,338005.0,0.031736,0.008751,IT-L23+ACA
MA0507.1,1.555035,1.717301e-20,3.086849e-19,18.510485,570.0,34469.0,3588.0,337401.0,0.016268,0.010522,IT-L23+ACA
MA0607.1,2.744364,0.000000e+00,0.000000e+00,300.000000,3157.0,31882.0,11875.0,329114.0,0.090100,0.034825,IT-L23+ACA
MA0619.1,1.551563,1.114389e-143,7.284055e-142,141.137627,4857.0,30182.0,32043.0,308946.0,0.138617,0.093971,IT-L23+ACA
MA0623.2,3.394161,1.042289e-161,7.494058e-160,159.125283,836.0,34203.0,2438.0,338551.0,0.023859,0.007150,IT-L23+ACA
MA0633.1,3.400680,5.048292e-223,7.259443e-221,220.139097,1159.0,33880.0,3396.0,337593.0,0.033077,0.009959,IT-L23+ACA
MA0668.1,4.431635,7.491126e-183,6.732649e-181,180.171814,695.0,34344.0,1550.0,339439.0,0.019835,0.004546,IT-L23+ACA
